<a href="https://colab.research.google.com/github/cheonghf/ML-P4-03/blob/main/Project_SourceCode_P4_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd

dataset_1 = 'https://raw.githubusercontent.com/cheonghf/ML-P4-03/refs/heads/main/bank_transactions_data_2.csv'

bt_df1 = pd.read_csv(dataset_1, index_col = 0)

#bt_df1.Channel.value_counts()

# Filter the dataframe for transactions where the Channel is 'Online'
df_atm = bt_df1[bt_df1['Channel'] == 'Online']

df_atm_trans_percentage =  (df_atm['TransactionAmount'] / (df_atm['TransactionAmount'] + df_atm['AccountBalance'])) * 100

df_atm_cleaned = df_atm[['AccountID','LoginAttempts','TransactionDuration']]

df_atm_cleaned['AtmTransactionPercentage'] = df_atm_trans_percentage #Insert Calculated AtmTransactionPercentage

df_atm_cleaned.set_index('AccountID')


<ipython-input-9-92c92c5bfc05>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_atm_cleaned['AtmTransactionPercentage'] = df_atm_trans_percentage #Insert Calculated AtmTransactionPercentage


,LoginAttempts,TransactionDuration,AtmTransactionPercentage
AccountID,,,
AC00019,1,56,10.114204
AC00070,1,25,2.107714
AC00411,1,198,0.180710
AC00150,1,59,0.233416
AC00459,1,173,11.055195
...,...,...,...
AC00216,1,169,5.413191
AC00069,1,26,5.427025
AC00047,1,40,0.902838
